# Symbolic Partial Derivative Routine

## Authors: Zach Etienne & Tyler Knowles

## This module contains a routine for computing partial derivatives of a mathematical expression that is written as several subexpressions.

**Notebook Status:** <font color='red'><b> In progress </b></font>

**TODO:**
1. Edit location of NRPy+ tutorial
1. Edit location of v3 Hamiltonian notebook (in introduction)
1. Include validation cell

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). Additionally, this notebook has been validated by checking that results are consistent with exact derivative expressions in [LALSuite](https://git.ligo.org/lscsoft/lalsuite).

### NRPy+ Source Code for this module: [SEOBNR_Derivative_Routine.py](../../edit/in_progress/SEOBNR/SEOBNR_Derivative_Routine.py)

## Introduction
$$\label{intro}$$

This notebook documents the symbolic partial derivative routine used to generate analytic derivatives of the [SEOBNRv3](https://git.ligo.org/lscsoft/lalsuite) Hamiltonian (documented [here](../Tutorial-SEOBNR_v3_Hamiltonian.ipynb)) and described in [this article](https://arxiv.org/abs/1803.06346).  In general, this notebook takes as input a file of inter-dependent mathematical expressions (in SymPy syntax), a file listing the names of values within those expressions, and a file listing all variables with which to take partial derivatives of each expression.  The output is a text file containing the original expression and those for each partial derivative computation.  The intention is to perform CSE on these expressions to create efficient partial derivative code!

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize core Python/NRPy+ modules
1. [Step 2:](#step2) Read in expressions
1. [Step 3:](#step3) Read in constants and variables
1. [Step 4:](#step4) List free symbols
1. [Step 5:](#step5) Convert expressions to function notation
1. [Step 6:](#step6) Differentiate with respect to xx
1. [Step 7:](#step7) Simplify derivative expressions
1. [Step 8:](#step8) Differentiate with respect to a specific free variable
1. [Step 9:](#step9) Output result
1. [Step 10:](#code_validation) Code Validation against `SEOBNR_Derivative_Routine` NRPy+ module
1. [Step 11:](#latex_pdf_output) Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from Python/NRPy+:

In [1]:
# Step 1.a: import all needed modules from Python/NRPy+:
import sympy as sp                # SymPy: The Python computer algebra package upon which NRPy+ depends
import os, sys                    # Standard Python modules for multiplatform OS-level functions

# Step 1.?: check system path so can use outputC; #TylerK: remove and put outputC back with other imports
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import superfast_uniq, lhrh      # Remove duplicate entries from a Python array; store left- and right-
                                              #   hand sides of mathematical expressions

<a id='step2'></a>

# Step 2: Read in expressions \[Back to [top](#toc)\]
$$\label{step2}$$

We read in the expressions of which we will compute partial derivatives in a single large string before splitting the string by line (carriage return) and by "=".  Doing so allows us to manipulate the right- and left-hand sides of the expressions appropriately.  We store the left- and right-hand sides in the array $\texttt{lr}$, which consists of $\texttt{lhrh}$ arrays with left-hand sides $\texttt{lhs}$ and right-hand sides $\texttt{rhs}$.  Note that $\texttt{Lambda}$ is a protected keyword in Python, so the variable $\Lambda$ in the Hamiltonian is renamed $\texttt{Lamb}$.

In [2]:
# Step 2.a: Read in expressions as a (single) string
with open('SEOBNR/Hamstring.txt', 'r') as file:
    expressions_as_lines = file.readlines()

# Step 2.b: Create and populate the "lr" array, which separates each line into left- and right-hand sides
#   Each entry is a string of the form lhrh(lhs='',rhs='')
lr = []

for i in range(len(expressions_as_lines)):
    # Ignore lines with 2 or fewer characters and those starting with #
    if len(expressions_as_lines[i]) > 2 and expressions_as_lines[i][0] != "#":
        # Split each line by its equals sign
        split_line = expressions_as_lines[i].split("=")
        # Append the line to "lr", removing spaces, "sp." prefixes, and replacing Lambda->Lamb
        #   (Lambda is a protected keyword):
        lr.append(lhrh(lhs=split_line[0].replace(" ","").replace("Lambda","Lamb"),
                       rhs=split_line[1].replace(" ","").replace("sp.","").replace("Lambda","Lamb")))

# Step 2.c: Separate and sympify right- and left-hand sides into separate arrays
lhss = []
rhss = []
for i in range(len(lr)):
    lhss.append(sp.sympify(lr[i].lhs))
    rhss.append(sp.sympify(lr[i].rhs))

<a id='step3'></a>

# Step 3: Read in constants and variables \[Back to [top](#toc)\]
$$\label{step3}$$

We read in and declare as SymPy symbols the constant values; derivatives with respect to these variables will be set to zero.  We then read in the variables with respect to which we want to take derivatives and declare those as SymPy variables as well.

In [3]:
# Step 3.a: Read in constants as a (single) string
with open('SEOBNR/Hamstring_constants.txt', 'r') as file:
    constants_as_lines = file.readlines()

# Step 3.b: Create "input_constants" array and populate with SymPy constants
input_constants = []
for constant in constants_as_lines:
    constant = sp.symbols(constant,real=True)
    input_constants.append(constant)

# Step 3.c: Read in variables with which to take derivatives
with open('SEOBNR/Hamstring_variables.txt', 'r') as file:
    variables_as_lines = file.readlines()

#Step 3.d: Create "dynamic_variables" array and populate with SymPy symbols
dynamic_variables = []
for variable in variables_as_lines:
    variable = sp.symbols(variable,real=True)
    dynamic_variables.append(variable)

<a id='step4'></a>

# Step 4: List free symbols \[Back to [top](#toc)\]
$$\label{step4}$$

By ''free symbols'' we mean the variables in the right-hand sides.  We first create a list of all such terms (using SymPy's built-in free_symbol attribute), including duplicates, and then strip the duplicates.  We then remove input constants from the symbol list.

In [4]:
# Step 4.a: Prepare array of "free symbols" in the right-hand side expressions
full_symbol_list_with_dups = []
for i in range(len(lr)):
    for variable in rhss[i].free_symbols:
        full_symbol_list_with_dups.append(variable)

# Step 4.b: Remove duplicate free symbols
full_symbol_list = superfast_uniq(full_symbol_list_with_dups)

# Step 4.c: Remove input constants from symbol list
for inputconst in input_constants:
    for symbol in full_symbol_list:
        if str(symbol) == str(inputconst):
            full_symbol_list.remove(symbol)

<a id='step5'></a>

# Step 5: Convert expressions to function notation \[Back to [top](#toc)\]
$$\label{step5}$$

In order to compute the partial derivative of each right-hand side, we mark each variable (left-hand side) and each free symbol (in right-hand sides) as a function with argument $\texttt{xx}$.

In [5]:
# Step 5.a: Convert each left-hand side to function notation
#   while separating and simplifying left- and right-hand sides
xx = sp.Symbol('xx',real=True)
func = []
for i in range(len(lr)):
    func.append(sp.sympify(sp.Function(lr[i].lhs)(xx)))

# Step 5.b: Mark each free variable as a function with argument xx
full_function_list = []
for symb in full_symbol_list:
    func = sp.sympify(sp.Function(str(symb))(xx))
    full_function_list.append(func)
    for i in range(len(rhss)):
        for var in rhss[i].free_symbols:
            if str(var) == str(symb):
                rhss[i] = rhss[i].subs(var,func)

<a id='step6'></a>

# Step 6: Differentiate with respect to xx \[Back to [top](#toc)\]
$$\label{step6}$$

Now we differentiate the right-hand expressions with respect to $\textrm{xx}$.  We use the SymPy $\texttt{diff}$ command, differentiating with respect to $\texttt{xx}$.  After so doing, we remove $\texttt{(xx)}$ and "Derivative" (which is output by $\texttt{diff}$), and use "prm" suffix to denote the derivative with respect to $\texttt{xx}$.

In [6]:
print(lhss[130],rhss[130])
# Zach says: Notice that the tortoise constant is being converted into a function:
print(str(sp.diff(rhss[130],xx)).replace("(xx)","").replace(", xx","prm").replace("Derivative",""))

csi2 (1/2 - 0.5*sign(1.5 - tortoise(xx)))*(csi(xx) - 1) + 1
(1/2 - 0.5*sign(1.5 - tortoise))*(csiprm) - 0.5*(csi - 1)*(sign(1.5 - tortoise)prm)


In [7]:
# Step 6: Use SymPy's diff function to differentiate right-hand sides with respect to xx
#   and append "prm" notation to left-hand sides
lhss_deriv = []
rhss_deriv = []
for i in range(len(rhss)):
    lhss_deriv.append(sp.sympify(str(lhss[i])+"prm"))
    print(i,lhss[i],rhss[i])
    print(sp.diff(rhss[i],xx))
    newrhs = sp.sympify(str(sp.diff(rhss[i],xx)).replace("(xx)","").replace(", xx","prm").replace("Derivative",""))
    rhss_deriv.append(newrhs)

0 sigmaKerr0 s1x(xx) + s2x(xx)
Derivative(s1x(xx), xx) + Derivative(s2x(xx), xx)
1 sigmaKerr1 s1y(xx) + s2y(xx)
Derivative(s1y(xx), xx) + Derivative(s2y(xx), xx)
2 sigmaKerr2 s1z(xx) + s2z(xx)
Derivative(s1z(xx), xx) + Derivative(s2z(xx), xx)
3 invm1m2 1/(m1(xx)*m2(xx))
-Derivative(m2(xx), xx)/(m1(xx)*m2(xx)**2) - Derivative(m1(xx), xx)/(m1(xx)**2*m2(xx))
4 m2overm1 invm1m2(xx)*m2(xx)**2
2*invm1m2(xx)*m2(xx)*Derivative(m2(xx), xx) + m2(xx)**2*Derivative(invm1m2(xx), xx)
5 m1overm2 invm1m2(xx)*m1(xx)**2
2*invm1m2(xx)*m1(xx)*Derivative(m1(xx), xx) + m1(xx)**2*Derivative(invm1m2(xx), xx)
6 sigmaStar0 m1overm2(xx)*s2x(xx) + m2overm1(xx)*s1x(xx)
m1overm2(xx)*Derivative(s2x(xx), xx) + m2overm1(xx)*Derivative(s1x(xx), xx) + s1x(xx)*Derivative(m2overm1(xx), xx) + s2x(xx)*Derivative(m1overm2(xx), xx)
7 sigmaStar1 m1overm2(xx)*s2y(xx) + m2overm1(xx)*s1y(xx)
m1overm2(xx)*Derivative(s2y(xx), xx) + m2overm1(xx)*Derivative(s1y(xx), xx) + s1y(xx)*Derivative(m2overm1(xx), xx) + s2y(xx)*Derivative(m1ov

49 xi2 1 - costheta(xx)**2
-2*costheta(xx)*Derivative(costheta(xx), xx)
50 xiUSCOREx e3USCOREy(xx)*nz(xx) - e3USCOREz(xx)*ny(xx)
e3USCOREy(xx)*Derivative(nz(xx), xx) - e3USCOREz(xx)*Derivative(ny(xx), xx) - ny(xx)*Derivative(e3USCOREz(xx), xx) + nz(xx)*Derivative(e3USCOREy(xx), xx)
51 xiUSCOREy -e3USCOREx(xx)*nz(xx) + e3USCOREz(xx)*nx(xx)
-e3USCOREx(xx)*Derivative(nz(xx), xx) + e3USCOREz(xx)*Derivative(nx(xx), xx) + nx(xx)*Derivative(e3USCOREz(xx), xx) - nz(xx)*Derivative(e3USCOREx(xx), xx)
52 xiUSCOREz e3USCOREx(xx)*ny(xx) - e3USCOREy(xx)*nx(xx)
e3USCOREx(xx)*Derivative(ny(xx), xx) - e3USCOREy(xx)*Derivative(nx(xx), xx) - nx(xx)*Derivative(e3USCOREy(xx), xx) + ny(xx)*Derivative(e3USCOREx(xx), xx)
53 vx ny(xx)*xiUSCOREz(xx) - nz(xx)*xiUSCOREy(xx)
ny(xx)*Derivative(xiUSCOREz(xx), xx) - nz(xx)*Derivative(xiUSCOREy(xx), xx) - xiUSCOREy(xx)*Derivative(nz(xx), xx) + xiUSCOREz(xx)*Derivative(ny(xx), xx)
54 vy -nx(xx)*xiUSCOREz(xx) + nz(xx)*xiUSCOREx(xx)
-nx(xx)*Derivative(xiUSCOREz(xx), xx) 

106 invdeltaT invdeltaTsqrtdeltaTsqrtdeltaR(xx)*sqrtdeltaR(xx)*sqrtdeltaT(xx)
invdeltaTsqrtdeltaTsqrtdeltaR(xx)*sqrtdeltaR(xx)*Derivative(sqrtdeltaT(xx), xx) + invdeltaTsqrtdeltaTsqrtdeltaR(xx)*sqrtdeltaT(xx)*Derivative(sqrtdeltaR(xx), xx) + sqrtdeltaR(xx)*sqrtdeltaT(xx)*Derivative(invdeltaTsqrtdeltaTsqrtdeltaR(xx), xx)
107 invsqrtdeltaT deltaTsqrtdeltaR(xx)*invdeltaTsqrtdeltaTsqrtdeltaR(xx)
deltaTsqrtdeltaR(xx)*Derivative(invdeltaTsqrtdeltaTsqrtdeltaR(xx), xx) + invdeltaTsqrtdeltaTsqrtdeltaR(xx)*Derivative(deltaTsqrtdeltaR(xx), xx)
108 invsqrtdeltaR deltaT(xx)*invdeltaTsqrtdeltaTsqrtdeltaR(xx)*sqrtdeltaT(xx)
deltaT(xx)*invdeltaTsqrtdeltaTsqrtdeltaR(xx)*Derivative(sqrtdeltaT(xx), xx) + deltaT(xx)*sqrtdeltaT(xx)*Derivative(invdeltaTsqrtdeltaTsqrtdeltaR(xx), xx) + invdeltaTsqrtdeltaTsqrtdeltaR(xx)*sqrtdeltaT(xx)*Derivative(deltaT(xx), xx)
109 w invLamb(xx)*ww(xx)
invLamb(xx)*Derivative(ww(xx), xx) + ww(xx)*Derivative(invLamb(xx), xx)
110 LambUSCOREr -a2(xx)*deltaTUSCOREr(xx)*xi2(xx) + 4*

SympifyError: Sympify of expression 'could not parse '(1/2 - 0.5*sign(1.5 - tortoise))*(csiprm) - 0.5*(csi - 1)*(sign(1.5 - tortoise)prm)'' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 1)

<a id='step7'></a>

# Step 7: Simplify derivative expressions \[Back to [top](#toc)\]
$$\label{step7}$$

We declare a function to simply the derivative expressions.  In particular, we want to remove terms equal to zero.

In [ ]:
# Step 7.a: Define derivative simplification function
def simplify_deriv(lhss_deriv,rhss_deriv):
    # Copy expressions into another array
    lhss_deriv_simp = []
    rhss_deriv_simp = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_simp.append(lhss_deriv[i])
        rhss_deriv_simp.append(rhss_deriv[i])
    # If a right-hand side is 0, substitute value 0 for the corresponding left-hand side in later terms
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == 0:
            for j in range(i+1,len(rhss_deriv_simp)):
                for var in rhss_deriv_simp[j].free_symbols:
                    if str(var) == str(lhss_deriv_simp[i]):
                        rhss_deriv_simp[j] = rhss_deriv_simp[j].subs(var,0)
    zero_elements_to_remove = []
    # Create array of indices for expressions that are zero
    for i in range(len(rhss_deriv_simp)):
        if rhss_deriv_simp[i] == sp.sympify(0):
            zero_elements_to_remove.append(i)

    # When removing terms that are zero, we need to take into account their new index (after each removal)
    count = 0
    for i in range(len(zero_elements_to_remove)):
        del lhss_deriv_simp[zero_elements_to_remove[i]+count]
        del rhss_deriv_simp[zero_elements_to_remove[i]+count]
        count -= 1
    return lhss_deriv_simp,rhss_deriv_simp

# Step 7.b: Call the simplication function and then copy results
lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv,rhss_deriv)
lhss_deriv = lhss_deriv_simp
rhss_deriv = rhss_deriv_simp

<a id='step8'></a>

# Step 8: Differentiate with respect to a specific free variable \[Back to [top](#toc)\]
$$\label{step8}$$

In [Step 6](#step6) we took a generic derivative of each term, assuming it is a function of the varible $\textrm{xx}$.  We now define a function that will select a specific free variable for differentiation.

In [ ]:
# Step 8.a: Define onevar derivative function
def deriv_onevar(lhss_deriv,rhss_deriv,variable_list,index):
    # Denote each variable with prm
    variableprm_list = []
    for variable in variable_list:
        variableprm_list.append(str(variable)+"prm")

    # Copy expressions into another array
    lhss_deriv_new = []
    rhss_deriv_new = []
    for i in range(len(rhss_deriv)):
        lhss_deriv_new.append(lhss_deriv[i])
        rhss_deriv_new.append(rhss_deriv[i])
    # For each free symbol's derivative, replace it with:
    #   1, if we are differentiating with respect to the variable, or
    #   0, if we are note differentiating with respect to that variable
    for i in range(len(rhss_deriv_new)):
        for var in variableprm_list:
            if variableprm_list.index(str(var))==index:
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,1)
            else:
                rhss_deriv_new[i] = rhss_deriv_new[i].subs(var,0)
    # Simplify derivative expressions again
    lhss_deriv_simp,rhss_deriv_simp = simplify_deriv(lhss_deriv_new,rhss_deriv_new)
    return lhss_deriv_simp,rhss_deriv_simp

# Step 8.b: Call the derivative function and populate dictionaries with the result
lhss_derivative = {}
rhss_derivative = {}
for index in range(len(dynamic_variables)):
    lhss_temp,rhss_temp = deriv_onevar(lhss_deriv,rhss_deriv,dynamic_variables,index)
    lhss_derivative[dynamic_variables[index]] = lhss_temp
    rhss_derivative[dynamic_variables[index]] = rhss_temp

<a id='step9'></a>

# Step 9: Output result \[Back to [top](#toc)\]
$$\label{step9}$$

We write the resulting derivatives in SymPy syntax.  Each partial derivative is output in its own file, in a similar format to the input expressions.

In [ ]:
# Step 9: Output original expression and each partial derivative expression in SymPy snytax
with open("partial_derivatives.txt", "w") as output:
    for i in range(len(lr)):
        right_side = lr[i].rhs
        right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("sign(","sp.sign(").replace("Abs(",
                                                "sp.Abs(").replace("Rational(","sp.Rational(")
        output.write(str(lr[i].lhs)+" = "+right_side_in_sp)
    for var in dynamic_variables:
        for i in range(len(lhss_derivative[var])):
            right_side = str(rhss_derivative[var][i])
            right_side_in_sp = right_side.replace("sqrt(","sp.sqrt(").replace("log(","sp.log(").replace("pi",
                                                "sp.pi").replace("sign(","sp.sign(").replace("Abs(",
                                                "sp.Abs(").replace("Rational(","sp.Rational(").replace("prm",
                                                "prm_"+str(var))
            output.write(str(lhss_derivative[var][i]).replace("prm","prm_"+str(var))+" = "+right_side_in_sp+"\n")

<a id='code_validation'></a>

# Step 10: Code Validation against `SEOBNR_Derivative_Routine` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$

<a id='latex_pdf_output'></a>

# Step 11: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-SEOBNR_Derivative_Routine.pdf](Tutorial-SEOBNR_Derivative_Routine.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [ ]:
import os,sys                    # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("NRPyPN_shortcuts",location_of_template_file=os.path.join(".."))